In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
}
players_url = []

# รายการ URL ของเว็บไซต์
for offset in range(0, 2881, 60):
    player_url = f"https://sofifa.com/players?type=all&lg%5B0%5D=13&lg%5B1%5D=16&lg%5B2%5D=19&lg%5B3%5D=31&lg%5B4%5D=53&offset={offset}"
    players_url.append(player_url)

# รายการเพื่อเก็บข้อมูลของนักเตะ
data = []

# ลูปเพื่อดึงข้อมูลจากแต่ละ URL
for player_url in players_url:
    # ส่งคำร้องขอ HTTP เพื่อดึงข้อมูลจาก URL ของนักเตะ
    resp = requests.get(player_url, headers=headers)

    # ตรวจสอบว่าเราได้หน้าเว็บที่ถูกต้องหรือไม่
    if resp.status_code != 200:
        print(f"เกิดข้อผิดพลาดในการดึงข้อมูลจาก URL: {player_url}")
        continue

    # ใช้ BeautifulSoup เพื่อแปลงเนื้อหา HTML เป็นวัตถุสำหรับการวิเคราะห์
    soup = BeautifulSoup(resp.content, 'html.parser')

    # ตรวจสอบเพื่อหา URL ของนักเตะ
    player_links = soup.select('td.col-name a')
    if not player_links:
        print("ไม่พบลิงค์ของนักเตะ")
    else:
        for player_link in player_links:
            player_url = "https://sofifa.com" + player_link["href"]
            # ตรวจสอบว่า URL ของนักเตะมีรูปแบบที่แน่นอน
            if player_url.startswith("https://sofifa.com/player/"):
                print("URL ของนักเตะ:", player_url)

                # ส่งคำร้องขอ HTTP เพื่อดึงข้อมูลจาก URL ของนักเตะ
                resp_player = requests.get(player_url, headers=headers)
                soup_player = BeautifulSoup(resp_player.content, 'lxml')

                # ตรวจสอบ ID และดึงข้อมูลอื่น ๆ ที่คุณต้องการ
                id_li = soup_player.select_one("li.ellipsis:has(label:-soup-contains('ID'))")
                if id_li:
                    Id = id_li.get_text(strip=True)
                    print(f"Id: {Id}")
                else:
                    print("ไม่พบข้อมูล Player ID")
                li_element = soup_player.select_one("li.ellipsis:has(label:-soup-contains('Release clause'))")
                if li_element:
                    span_element = li_element.find("span")
                    if span_element:
                        release_clause = span_element.get_text(strip=True)
                        print(release_clause)
                    else:
                        print("ไม่พบข้อมูล 'Release clause'")
                else:
                    print("ไม่พบข้อมูล 'Release clause'")
                preferred_foot_li = soup_player.select_one("li.ellipsis:has(label:-soup-contains('Preferred foot'))")
                if preferred_foot_li:
                    preferred_foot = preferred_foot_li.get_text(strip=True).replace("Preferred foot", "")
                    print(f"Preferred foot: {preferred_foot}")
                else:
                    print("ไม่พบข้อมูล 'preferred foot'")
                work_rate_li = soup_player.select_one("li.ellipsis:has(label:-soup-contains('Work rate'))")
                if work_rate_li:
                    span_element = work_rate_li.find("span")
                    if span_element:
                        work_rate = span_element.get_text(strip=True)
                        print(f"Work rate: {work_rate}")
                    else:
                        print("ไม่พบข้อมูล Work rate")
                else:
                    print("ไม่พบข้อมูล Work rate")
                body_type_li = soup_player.select_one("li.ellipsis:has(label:-soup-contains('Body type'))")
                if body_type_li:
                    span_element = body_type_li.find("span")
                    if span_element:
                        body_type = span_element.get_text(strip=True)
                        print(f"Body type: {body_type}")
                    else:
                        print("ไม่พบข้อมูล Body type")
                else:
                    print("ไม่พบข้อมูล Body type")
                img_tag = soup_player.select_one('body > div:nth-child(5) > div > div.col.col-12 > div:nth-child(4) > div > h5 > a > img')

                if img_tag:
        # ดึงข้อมูล title จาก attribute 'title' ของ tag <img>
                    title = img_tag.get('title')
                    print('Title:', title)
                else:
                    print('ไม่พบ tag <img> ที่ระบุ')

        
                
                

                # ตรวจสอบคุณสมบัติอื่น ๆ และดึงข้อมูลเพื่อนำมาแสดง
                property_names = [
                    "Best overall","Crossing","Finishing","Heading accuracy",
                    "Short passing","Volleys","Dribbling","Curve","FK Accuracy","Long passing","Ball control",
                    "Acceleration","Sprint speed","Agility","Reactions","Balance","Shot power","Jumping","Stamina",
                    "Strength","Long shots","Aggression","Interceptions","Positioning","Vision","Penalties",
                    "Composure","Defensive awareness","Standing tackle","Sliding tackle","GK Diving","GK Handling",
                    "GK Kicking","GK Positioning","GK Reflexes"
                ]

                span_tags = soup_player.select('div:nth-child(6) span.bp3-tag.p')

                player_data = {
                    "ID": Id,
                    "Release clause": release_clause,
                    "prefer_foot": preferred_foot,
                    "Work rate": work_rate,
                    "Body type": body_type,
                    "league" : title,
                    
                    
                }

                for i, span_tag in enumerate(span_tags):
                    text = span_tag.get_text(strip=True)
                    property_name = property_names[i] if i < len(property_names) else "N/A"
                    player_data[property_name] = text

                data.append(player_data)

# สร้าง DataFrame จากข้อมูลที่เก็บไว้
df2 = pd.DataFrame(data)

# แสดง DataFrame



URL ของนักเตะ: https://sofifa.com/player/192985/kevin-de-bruyne/240014/
Id: ID192985
€190.6M
Preferred foot: Right
Work rate: High/ Medium
Body type: Unique
Title: England
URL ของนักเตะ: https://sofifa.com/player/239085/erling-haaland/240014/
Id: ID239085
€356.1M
Preferred foot: Left
Work rate: High/ Medium
Body type: Unique
Title: England
URL ของนักเตะ: https://sofifa.com/player/231747/kylian-mbappe/240014/
Id: ID231747
€349.4M
Preferred foot: Right
Work rate: High/ Low
Body type: Unique
Title: France
URL ของนักเตะ: https://sofifa.com/player/202126/harry-kane/240014/
Id: ID202126
€197.2M
Preferred foot: Right
Work rate: High/ High
Body type: Unique
Title: Germany
URL ของนักเตะ: https://sofifa.com/player/188545/robert-lewandowski/240014/
Id: ID188545
€118.9M
Preferred foot: Right
Work rate: High/ Medium
Body type: Unique
Title: Spain
URL ของนักเตะ: https://sofifa.com/player/192119/thibaut-courtois/240014/
Id: ID192119
€129.2M
Preferred foot: Left
Work rate: Medium/ Medium
Body type: Un

In [4]:
df2

,ID,Release clause,prefer_foot,Work rate,Body type,league,Best overall,Crossing,Finishing,Heading accuracy,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,ID192985,€190.6M,Right,High/ Medium,Unique,England,91,95,85,55,...,83,88,66,70,53,15,13,5,10,13
1,ID239085,€356.1M,Left,High/ Medium,Unique,England,93,47,96,83,...,86,87,38,47,29,7,14,13,11,7
2,ID231747,€349.4M,Right,High/ Low,Unique,France,93,78,94,73,...,84,88,26,34,32,13,5,7,11,6
3,ID202126,€197.2M,Right,High/ High,Unique,Germany,90,80,95,89,...,92,92,46,46,38,8,10,11,14,11
4,ID188545,€118.9M,Right,High/ Medium,Unique,Spain,90,76,93,91,...,90,88,35,42,19,15,6,12,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,ID264240,€154.1M,Right,High/ High,Normal (170-185),Spain,86,69,71,42,...,51,74,59,78,72,8,10,11,7,13
2913,ID268438,€25.3M,Right,High/ Medium,Lean (170-185),England,75,68,70,44,...,56,73,28,37,29,6,15,14,13,7
2914,ID253072,€89.5M,Right,High/ High,Normal (170-185),England,85,68,81,79,...,78,79,39,43,34,14,10,12,10,13
2915,ID253163,€197.6M,Right,Medium/ High,Normal (185+),Spain,88,58,56,85,...,43,76,86,86,86,11,14,9,11,8


In [5]:
df2.to_csv('info2022.csv', index = False )